In [1]:

import os
import sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(''))) + '/src')
from config import *

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from functools import reduce

experiment = "REACTION"

df_event = pd.read_parquet(PREPROCESSED_DIR / f"{experiment}_events.pq")
df_sample = (pd.read_parquet(PREPROCESSED_DIR / f'{experiment}_samples.pq')
 .sort_values(["experiment", "participant_id", "trial_id","time"])
)


In [2]:

def rename_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Renames columns by joining multi-level column names with different delimiters."""
    # Iterate over all column names
    df.columns = [f"{col[0]}" if col[1] == '' else f"{col[0]}_{col[1]}" for col in df.columns.values]
    return df

# Event features

In [3]:


def get_pre_calculated_metrics_feature(df: pd.DataFrame) -> pd.DataFrame:
    """
    Returns pd.Dataframe with columns ['experiment','participant_id', X_FEATURES],
    where X_FEATURES is a collection of features found by the following cartesian product:
    {'peak_velocity', 'amplitude', 'duration', 'avg_pupil_size'} x {np.mean, np.min, np.max, np.median, np.std}
    """
    logging.info("Extracting precalculated metrics")
    features_df = (df
    .groupby(["experiment", "participant_id"])
    .agg(
        mean_peak_velocity_sacc = ('peak_velocity', lambda x: x[df.loc[x.index, 'event'] == 'ESACC'].mean()),
        mean_amplitude_sacc = ('amplitude', lambda x: x[df.loc[x.index, 'event'] == 'ESACC'].mean()),
        mean_duration_sacc = ('duration', lambda x: x[df.loc[x.index, 'event'] == 'ESACC'].mean()),
        mean_duration_fix = ('duration', lambda x: x[df.loc[x.index, 'event'] == 'EFIX'].mean()),
        mean_pupil_size_fix = ('avg_pupil_size', lambda x: x[df.loc[x.index, 'event'] == 'EFIX'].mean()),
    )
    .reset_index()
    )    
    return features_df




In [4]:
def add_saccade_direction(df: pd.DataFrame) -> pd.DataFrame:
    logging.info("Adding saccade direction")
    df = (df
        .assign(saccade_direction_x = lambda x: np.select([(x["event"] == 'ESACC') & (np.abs(x["end_x"] - x["start_x"]) < 50),
                                                        (x["event"] == 'ESACC') & (x["end_x"] > x["start_x"]),
                                                        (x["event"] == 'ESACC') & (x["end_x"] < x["start_x"])],
                                                        ['no_direction',"right", "left"], default=None))
        .assign(saccade_direction_y = lambda x: np.select([(x["event"] == 'ESACC') & (np.abs(x["end_y"] - x["start_y"]) < 50),
                                                        (x["event"] == 'ESACC') & (x["end_y"] > x["start_y"]),
                                                        (x["event"] == 'ESACC') & (x["end_y"] < x["start_y"])],
                                                        ['no_direction',"up", "down"], default=None))
        .assign(saccade_direction = lambda x: np.select([(x["saccade_direction_x"].isin(["right", "left"])) & (x["saccade_direction_y"].isin(["up", "down"]))
                                                         , (x["saccade_direction_x"].isin(["right", "left"])) & (x["saccade_direction_y"] == "no_direction")
                                                         , (x["saccade_direction_x"] == "no_direction") & (x["saccade_direction_y"].isin(["down", "up"]))
                                                         , (x["saccade_direction_x"] == "no_direction") & (x["saccade_direction_y"] == "no_direction")
                                                        ]
                                                        , [(x["saccade_direction_x"] + "_" + x["saccade_direction_y"])
                                                         , (x["saccade_direction_x"])
                                                         , (x["saccade_direction_y"])
                                                         , 'no_direction'
                                                        ]
                                                        , default=None))
    )
    return df


def add_stimulus_direction(df: pd.DataFrame) -> pd.DataFrame:
    logging.info("Adding stimulus direction")

    MIDDLE_FIXPOINT_X=960
    MIDDLE_FIXPOINT_Y=540
    
    df = (df
        .assign(stimulus_direction_x = lambda x: np.select([(x["event"] == 'FIXPOINT') & (np.abs(x["stimulus_x"] - MIDDLE_FIXPOINT_X) < 50),
                                                        (x["event"] == 'FIXPOINT') & (x["stimulus_x"] > MIDDLE_FIXPOINT_X),
                                                        (x["event"] == 'FIXPOINT') & (x["stimulus_x"] < MIDDLE_FIXPOINT_X)],
                                                        ["middle", "right", "left"], default=None))
        .assign(stimulus_direction_y = lambda x: np.select([(x["event"] == 'FIXPOINT') & (np.abs(x["stimulus_y"] - MIDDLE_FIXPOINT_Y) < 50),
                                                        (x["event"] == 'FIXPOINT') & (x["stimulus_y"] > MIDDLE_FIXPOINT_Y),
                                                        (x["event"] == 'FIXPOINT') & (x["stimulus_y"] < MIDDLE_FIXPOINT_Y)],
                                                        ['middle',"up", "down"], default=None))
        .assign(stimulus_direction = lambda x: np.select([(x["stimulus_direction_x"].isin(["right", "left"])) & (x["stimulus_direction_y"].isin(["up", "down"]))
                                                         , (x["stimulus_direction_x"].isin(["right", "left"])) & (x["stimulus_direction_y"] == "middle")
                                                         , (x["stimulus_direction_x"] == "middle") & (x["stimulus_direction_y"].isin(["right", "left"]))
                                                         , (x["stimulus_direction_x"] == "middle") & (x["stimulus_direction_y"] == "middle")
                                                        ]
                                                        , [(x["stimulus_direction_x"] + "_" + x["stimulus_direction_y"])
                                                         , (x["stimulus_direction_x"])
                                                         , (x["stimulus_direction_x"])
                                                         , 'middle'
                                                        ]
                                                        , default=None)).ffill()
    )
    return df

def get_trial_correctness_df(df:pd.DataFrame) -> pd.DataFrame:
    logging.info("Extracting trial correctness")

    df_trials = (df_event
            .query('stimulus_active == True')
            .sort_values(by=["participant_id", "trial_id","time"])
            .assign(stimulus_time = lambda x: np.select([x.event == "FIXPOINT", x.event != "FIXPOINT"], [x.time, None]))
            .assign(stimulus_time = lambda x: x["stimulus_time"].ffill())
            .pipe(add_stimulus_direction)
            .pipe(add_saccade_direction)
            .assign(is_saccade_correct = lambda x: np.select([ (x["saccade_direction"].notna()) & (x["saccade_direction"] == 'no_direction')
                                                            , (x["saccade_direction"].notna()) & (x["saccade_direction"] == x["stimulus_direction"])
                                                            , (x["saccade_direction"].notna()) & (x["saccade_direction"] != x["stimulus_direction"])
                                                            ],
                                                            [False, True, False], default=None)) 
            .assign(is_trial_correct = lambda x: (
                    x.sort_values(by=["participant_id", "trial_id", "time"])
                    .groupby(["participant_id", "trial_id"])["is_saccade_correct"]
                    .transform(lambda group: (
                        True if not group.dropna().empty and group.dropna().iloc[0] == True else
                        False if not group.dropna().empty and group.dropna().iloc[0] == False else
                        None
                    ))
                ))
    )
    return df_trials

In [5]:

def get_n_correct_trials_feature(df: pd.DataFrame) -> pd.DataFrame:
    """
    Returns pd.Dataframe with columns ['experiment', 'participant_id', 'n_correct_trials']
    """
    logging.info("Extracting n correct trials")

    feature_df = (df
     .pipe(get_trial_correctness_df)
     .groupby(["experiment","participant_id", "trial_id"])
     .agg(is_trial_correct = ('is_trial_correct', 'min')) 
     .reset_index()
     .groupby(["experiment", "participant_id"])
     .agg(n_correct_trials = ('is_trial_correct', 'sum'))
     .reset_index()
    [["experiment", "participant_id", "n_correct_trials"]]
    )
    
    return feature_df


def get_prop_trials_feature(df: pd.DataFrame) -> pd.DataFrame:
    """
    Returns pd.Dataframe with columns ['experiment', 'participant_id', 'prop_correct_trials']
    """
    logging.info("Extracting prop correct trials")

    feature_df = (df
     .pipe(get_trial_correctness_df)
     .groupby(["experiment","participant_id", "trial_id"])
     .agg(is_trial_correct = ('is_trial_correct', 'min')) 
     .reset_index()
     .groupby(["experiment", "participant_id"])
     .agg(n_correct_trials = ('is_trial_correct', 'sum'),
          n_trials = ('is_trial_correct', 'count'))
     .reset_index()
     .assign(prop_correct_trials = lambda x: x["n_correct_trials"] / x["n_trials"])
     [["experiment", "participant_id", "prop_correct_trials"]]
    )
    
    return feature_df

def get_reaction_time_feature(df: pd.DataFrame) -> pd.DataFrame:
    logging.info("Extracting reaction time")

    return (df
        .query('stimulus_active == True')
        .pipe(get_trial_correctness_df)
        .sort_values(by=["participant_id", "trial_id", "time"])
        .assign(is_saccade_correct = lambda x: np.select([ (x["is_saccade_correct"] == True) ], [True], default=None))
        .query("is_saccade_correct == True")
        .groupby(["experiment","participant_id", "trial_id"])
        .first()
        .reset_index()
        .assign(reaction_time = lambda group: group["start_time"] - group["stimulus_time"])
        .groupby(["experiment", "participant_id"])
        .agg(reaction_time_avg = ('reaction_time', 'mean'),
             reaction_time_std = ('reaction_time', 'std'))
        .reset_index()
    )

         

# Sample features

In [ ]:
def get_acceleration_feature(df: pd.DataFrame) -> pd.DataFrame:
    """Finds acceleration features for anti saccade experiment

    Args:
        df (pd.DataFrame): Dataframe with raw samples

    Returns:
        pd.DataFrame: Dataframe with columns ['experiment','participant_id', X_FEATURES]
        where X_FEATURES is a collection of features found by the following cartesian product:
        {'total_acceleration_magnitude_left', 'total_acceleration_magnitude_right'} x {np.mean, np.min, np.max, np.median, np.std}
    """
    logging.info("Extracting acceleration")
    acceleration = (df.join((df
    .groupby(["experiment", "participant_id", "trial_id"])[['x_velocity_left', 'y_velocity_left', 'x_velocity_right', 'y_velocity_right']].shift(1)
    .rename(columns={'x_velocity_left': 'x_velocity_left_lagged'
            , 'y_velocity_left': 'y_velocity_left_lagged'
            , 'x_velocity_right': 'x_velocity_right_lagged'
            , 'y_velocity_right': 'y_velocity_right_lagged'}))
    ).assign(x_acceleration_left = lambda x: (x["x_velocity_left"] - x["x_velocity_left_lagged"]) / (1/2000),
            y_acceleration_left = lambda x: (x["y_velocity_left"] - x["y_velocity_left_lagged"]) / (1/2000),
            x_acceleration_right = lambda x: (x["x_velocity_right"] - x["x_velocity_right_lagged"]) / (1/2000),
            y_acceleration_right = lambda x: (x["y_velocity_right"] - x["y_velocity_right_lagged"]) / (1/2000))
    .assign(total_acceleration_magnitude_left = lambda x: np.sqrt( np.power(x["x_acceleration_left"], 2) + np.power(x["y_acceleration_left"], 2)),
            total_acceleration_magnitude_right = lambda x: np.sqrt( np.power(x["x_acceleration_right"], 2) + np.power(x["y_acceleration_right"], 2)))
    .groupby(["experiment", "participant_id"])
    .agg({'total_acceleration_magnitude_left': [np.mean, np.min, np.max, np.median, np.std],
        'total_acceleration_magnitude_right': [np.mean, np.min, np.max, np.median, np.std]
        })
    .reset_index()
    .pipe(rename_columns)
    )
    return acceleration


# Eye disconjugacy
# Paper: https://www.liebertpub.com/doi/full/10.1089/neu.2014.3687

def get_disconjugacy_feature(df:pd.DataFrame) -> pd.DataFrame:
    logging.info("Extracting disconjugacy")
    disconjugacy = (df
        .sort_values(["experiment", "participant_id", "trial_id", "time"])
        .query("x_left == x_left & x_right == x_right & y_left == y_left & y_right == y_right") # same as not null
        .groupby(["experiment", "participant_id"])
        .apply(lambda group: group.assign(
            x_left_rolling=group["x_left"].rolling(window=5, min_periods=1).mean(),
            x_right_rolling=group["x_right"].rolling(window=5, min_periods=1).mean(),
            y_left_rolling=group["y_left"].rolling(window=5, min_periods=1).mean(),
            y_right_rolling=group["y_right"].rolling(window=5, min_periods=1).mean()
        ))
        .reset_index(drop=True)
        .assign(
            X_diffs = lambda x: ((x["x_left_rolling"] - x["x_right_rolling"]) - 0)**2,
            Y_diffs = lambda x: ((x["y_left_rolling"] - x["y_right_rolling"]) - 0)**2
        )
        .groupby(["experiment", "participant_id"])
        .apply(lambda group: group.assign(
            X_squared_scaled = group["X_diffs"] / group.shape[0],
            Y_squared_scaled = group["Y_diffs"] / group.shape[0]
        ))
        .reset_index(drop=True)
        .groupby(["experiment", "participant_id"])
        .agg(
            Var_X = ("X_squared_scaled", "sum"),
            Var_Y = ("Y_squared_scaled", "sum")
        )
        .assign(
            Var_total = lambda x: x["Var_X"] + x["Var_Y"]
        )
        .reset_index()
        [["experiment", "participant_id", "Var_total"]]
    )
    return disconjugacy



# Combine

In [7]:
def get_reaction_features(df_event: pd.DataFrame, df_sample:pd.DataFrame) -> pd.DataFrame:
    """Runs all reaction feature extractions

    Args:
        df (pd.DataFrame): The preprocessed dataframe

    Returns:
        pd.DataFrame: Dataframe with columns ["experiment", "participant_id", X_FEATURES], where X_FEATURES is a collection of features
    """
    logging.info("Extracting reaction features")
    event_feature_functions = [get_n_correct_trials_feature, get_prop_trials_feature, get_reaction_time_feature]
    df_event_features_list = [f(df=df_event) for f in event_feature_functions]
    
    sample_feature_functions = [get_acceleration_feature, get_disconjugacy_feature]
    df_sample_features_list = [f(df=df_sample) for f in sample_feature_functions]
    
    df_features_list = df_event_features_list + df_sample_features_list
    
    df_features = reduce(lambda x, y: pd.merge(x, y, on = ["experiment", "participant_id"]), df_features_list)
    
    return df_features


features = get_reaction_features(df_event=df_event, df_sample=df_sample)
    
    

2025-04-16 11:39:09,648 - INFO - 417567659.get_reaction_features:10 - Extracting reaction features
2025-04-16 11:39:09,649 - INFO - 2577773216.get_n_correct_trials_feature:5 - Extracting n correct trials
2025-04-16 11:39:09,650 - INFO - 494265785.get_trial_correctness_df:57 - Extracting trial correctness
2025-04-16 11:39:09,738 - INFO - 494265785.add_stimulus_direction:28 - Adding stimulus direction
2025-04-16 11:39:09,785 - INFO - 494265785.add_saccade_direction:2 - Adding saccade direction
2025-04-16 11:39:10,167 - INFO - 2577773216.get_prop_trials_feature:25 - Extracting prop correct trials
2025-04-16 11:39:10,167 - INFO - 494265785.get_trial_correctness_df:57 - Extracting trial correctness
2025-04-16 11:39:10,192 - INFO - 494265785.add_stimulus_direction:28 - Adding stimulus direction
2025-04-16 11:39:10,236 - INFO - 494265785.add_saccade_direction:2 - Adding saccade direction
2025-04-16 11:39:10,633 - INFO - 2577773216.get_reaction_time_feature:43 - Extracting reaction time
2025-0

# Save

In [8]:
features.to_parquet(FEATURES_DIR / f"{experiment}_features.pq")


# How much faster is polars?

In [9]:
import polars as pl
def get_acceleration_feature_pl(df: pl.DataFrame) -> pl.DataFrame:
    """
    Finds acceleration features for anti saccade experiment using Polars.

    Args:
        df (pl.DataFrame): Dataframe with raw samples.

    Returns:
        pl.DataFrame: Dataframe with columns ['experiment','participant_id', X_FEATURES]
        where X_FEATURES is a collection of features found by the following cartesian product:
        {'total_acceleration_magnitude_left', 'total_acceleration_magnitude_right'} 
        x {mean, min, max, median, std}
    """
    logging.info("Started acceleration feature")

    df = df.with_columns([
        pl.col("x_velocity_left").shift(1).over(["experiment", "participant_id", "trial_id"]).alias("x_velocity_left_lagged"),
        pl.col("y_velocity_left").shift(1).over(["experiment", "participant_id", "trial_id"]).alias("y_velocity_left_lagged"),
        pl.col("x_velocity_right").shift(1).over(["experiment", "participant_id", "trial_id"]).alias("x_velocity_right_lagged"),
        pl.col("y_velocity_right").shift(1).over(["experiment", "participant_id", "trial_id"]).alias("y_velocity_right_lagged")
    ]).with_columns([
        ((pl.col("x_velocity_left") - pl.col("x_velocity_left_lagged")) * 2000).alias("x_acceleration_left"),
        ((pl.col("y_velocity_left") - pl.col("y_velocity_left_lagged")) * 2000).alias("y_acceleration_left"),
        ((pl.col("x_velocity_right") - pl.col("x_velocity_right_lagged")) * 2000).alias("x_acceleration_right"),
        ((pl.col("y_velocity_right") - pl.col("y_velocity_right_lagged")) * 2000).alias("y_acceleration_right")
    ]).with_columns([
        (pl.col("x_acceleration_left").pow(2) + pl.col("y_acceleration_left").pow(2)).sqrt().alias("total_acceleration_magnitude_left"),
        (pl.col("x_acceleration_right").pow(2) + pl.col("y_acceleration_right").pow(2)).sqrt().alias("total_acceleration_magnitude_right")
    ])

    acceleration = df.group_by(["experiment", "participant_id"]).agg([
        pl.col("total_acceleration_magnitude_left").mean().alias("total_acceleration_magnitude_left_mean"),
        pl.col("total_acceleration_magnitude_left").min().alias("total_acceleration_magnitude_left_min"),
        pl.col("total_acceleration_magnitude_left").max().alias("total_acceleration_magnitude_left_max"),
        pl.col("total_acceleration_magnitude_left").median().alias("total_acceleration_magnitude_left_median"),
        pl.col("total_acceleration_magnitude_left").std().alias("total_acceleration_magnitude_left_std"),

        pl.col("total_acceleration_magnitude_right").mean().alias("total_acceleration_magnitude_right_mean"),
        pl.col("total_acceleration_magnitude_right").min().alias("total_acceleration_magnitude_right_min"),
        pl.col("total_acceleration_magnitude_right").max().alias("total_acceleration_magnitude_right_max"),
        pl.col("total_acceleration_magnitude_right").median().alias("total_acceleration_magnitude_right_median"),
        pl.col("total_acceleration_magnitude_right").std().alias("total_acceleration_magnitude_right_std")
    ])

    return acceleration

In [10]:
get_acceleration_feature_pl(pl.from_dataframe(df_sample))

2025-04-16 11:30:25,935 - INFO - 576518341.get_acceleration_feature_pl:15 - Started acceleration feature


experiment,participant_id,total_acceleration_magnitude_left_mean,total_acceleration_magnitude_left_min,total_acceleration_magnitude_left_max,total_acceleration_magnitude_left_median,total_acceleration_magnitude_left_std,total_acceleration_magnitude_right_mean,total_acceleration_magnitude_right_min,total_acceleration_magnitude_right_max,total_acceleration_magnitude_right_median,total_acceleration_magnitude_right_std
str,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""REACTION""",228,1168.504499,0.0,80498.44719,824.621125,1848.183254,1253.00462,0.0,72089.94382,894.427191,2234.827616
"""REACTION""",250,2001.069687,0.0,839214.037061,1000.0,15637.75885,2862.702345,0.0,699117.186171,1077.032961,12050.559675
"""REACTION""",389,5517.936435,0.0,450317.310349,3984.971769,6144.386595,1871.424492,0.0,68170.374797,1442.22051,1713.942293
"""REACTION""",382,1302.536136,0.0,91988.260121,721.110255,2916.573101,1504.203617,0.0,52803.408981,1077.032961,1875.208434
"""REACTION""",336,1749.268286,0.0,100428.681162,1341.640786,2575.02595,1816.560171,0.0,152000.526315,1341.640786,3460.292738
…,…,…,…,…,…,…,…,…,…,…,…
"""REACTION""",257,1526.154421,0.0,66616.814694,1000.0,3156.156059,1166.968774,0.0,67656.189665,632.455532,2942.709032
"""REACTION""",378,1334.683534,0.0,132466.146619,721.110255,3727.230997,1355.79568,0.0,127422.60396,848.528137,2690.572802
"""REACTION""",180,1037.317366,0.0,93955.308525,800.0,1603.040469,1175.904648,0.0,61691.490499,894.427191,1553.632166


In [11]:
get_acceleration_feature(df_sample)

2025-04-16 11:30:33,309 - INFO - 654501450.get_acceleration_feature:12 - Extracting acceleration


KeyboardInterrupt: 